# <font color='#000FF'> **CDeep3M2** </font>


In [0]:
#@title # <==== Click 'Run Cell' for CDeep3M Installation
#@markdown (Time est.: 20-30min)

#@markdown A hardware check will initially performed to ensure the 
#@markdown runtime envirnment has sufficient GPU vRAM, 
#@markdown otherwise please click: Runtime -> Factory Reset
#################
# Prep
#################

!mkdir -p /home/
%cd /home/
# Get the repo
!git clone https://github.com/haberlmatt/cdeep3m-colab.git

# Install CDeep3M - Packaged into a installer file
!chmod 777 /home/cdeep3m-colab/*

# Make sure enough GPU memory available on this instance
!python /home/cdeep3m-colab/tests/pythontest_GPUmem.py
#!nvidia-smi

##################################
# Run CDeep3M installation
##################################
!/bin/bash /home/cdeep3m-colab/CDeep3M_V2_colab-installer.sh > /home/CDeep3m_installation.log

#################
# Set Environment
#################
import os
os.environ['PATH'] += ":/home/nd_sense/caffe_nd_sense_segmentation/"
os.environ['CAFFE_PATH'] = "/home/nd_sense/caffe_nd_sense_segmentation/"
os.environ['PYTHONPATH'] = "/home/nd_sense/caffe_nd_sense_segmentation/distribute/python/"
os.environ['PATH'] += ":/home/cdeep3m/"

print("CDeep3M installation completed \n   Runnning CDeep3M Version")
!cat /home/cdeep3m/VERSION
print("\nPlease see https://github.com/haberlmatt/cdeep3m-colab for further information on using CDeep3M")

/home
Cloning into 'cdeep3m-colab'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 137 (delta 79), reused 9 (delta 7), pack-reused 0
Receiving objects: 100% (137/137), 35.11 KiB | 473.00 KiB/s, done.
Resolving deltas: 100% (79/79), done.
python3: can't open file '/home/cdeep3m-colab/pythontest_GPUmem.py': [Errno 2] No such file or directory
Extracting templates from packages: 100%
Cloning into 'bats-core'...
remote: Enumerating objects: 2102, done.
remote: Total 2102 (delta 0), reused 0 (delta 0), pack-reused 2102
Receiving objects: 100% (2102/2102), 598.42 KiB | 1.54 MiB/s, done.
Resolving deltas: 100% (1147/1147), done.
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.4.0 which is incompatible.
sed: can't read /usr/bin/pip3: No such file or directory
Cloning into 'caffe_nd_sense_segmentation'...
remote: Enumerating objects: 26205, done.
remote: Total 26205

In [5]:
!ls /home

CDeep3m_installation.log


In [0]:
#@title # CDeep3M Colab -- Retrain model GUI
#@markdown Simple user interface to run CDeep3M on Colab
#@markdown Please make sure to run installation above first

#@markdown ### Please pick a DOI from the CDeep3M model zoo
#@markdown #### See: http://www.cellimagelibrary.org/cdeep3m
Model_DOI = 'https://doi.org/10.7295/W9CDEEP3M50687' #@param {type:"string"}
#@markdown ### Insert Path to your training images here:
Image_Path = '/home/cdeep3m/mito_testsample/training/images' #@param {type:"string"}
#@markdown ### Insert Path to your training labels here:
Label_Path = '/home/cdeep3m/mito_testsample/training/labels' #@param {type:"string"}

secondary_augm=int(-1)
tertiary_augm=0
# Hidden feature, uncomment following two lines to set secondary and tertiary augmentations:
#secondary_augm = -1 #@param {type:"slider", min:-1, max:10, step:1}
#tertiary_augm = 0 #@param {type:"slider", min:0, max:10, step:1}


#@markdown ### How many training iterations:
num_iter = 1000 #@param {type:"slider", min:100, max:5000, step:100}
#@markdown ### Train network seeing 1 frame, 3 frames, 5 frames:
train_1fm = True #@param {type:"boolean"}
train_3fm = True #@param {type:"boolean"}
train_5fm = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Enter a path where files are written:
output_path = "/home/retrained_model/" #@param {type:"string"}
#@markdown ---

import os
if Model_DOI is not None and Model_DOI !='':    
    %cd /home/    
    !rm -r /home/trainedmodel
    !mkdir /home/trainedmodel/
    cmd=str("wget -O model_download " + Model_DOI)
    print(cmd)   
    os.system(cmd)
    #!wget -O "model_download" Model_DOI
    !tar -xvf model_download -C /home/trainedmodel/
    d = '/home/trainedmodel/'
    model_path = [os.path.join(d, o) for o in os.listdir(d) 
                        if os.path.isdir(os.path.join(d,o))]
else:
    print('Provide a model')

# Convert into 1fm, 3fm and 5fm selection into string
fm_list = [str('1fm' * train_1fm), str('3fm' * train_3fm), str('5fm' * train_5fm)]
filtered_fm = filter(lambda x: x != "", fm_list)
combined_fm = list(filtered_fm)
fm_string = ','.join(combined_fm)
if fm_string is not None and fm_string !='':
    print('Running: ' + fm_string)
else:
    print('Provide a selection for which fm model to run')

print('Starting to run ...')
# Preprocess Training Data
augmented_path='/home/augmented_data/'
exec_command=str('PreprocessTraining.py ' + Image_Path + ' ' + Label_Path + ' ' + str(secondary_augm) + ' ' + str(tertiary_augm) + ' ' + augmented_path) 
print(exec_command)
!{exec_command}
# currently removed --models ' + fm_string + ' '
exec_command=str('runtraining.sh ' + '--numiterations ' + str(num_iter)  + ' ' + augmented_path +  ' ' + output_path) 
print(exec_command)
!{exec_command}

if train_1fm == True:
    exec_command=str('PlotValidation.py ' + os.path.join(output_path, '1fm','log'))
    !{exec_command}
if train_3fm == True:
    exec_command=str('PlotValidation.py ' + os.path.join(output_path, '3fm','log'))
    !{exec_command}
if train_3fm == True:
    exec_command=str('PlotValidation.py ' + os.path.join(output_path, '5fm','log'))
    !{exec_command}